In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from fuel.datasets.cifar10 import CIFAR10
from fuel.transformers import ScaleAndShift, Cast, Flatten
from fuel.streams import DataStream
from fuel.schemes import SequentialScheme, ShuffledScheme

CIFAR10.default_transformers = (
    (ScaleAndShift, [2.0 / 255.0, -1], {'which_sources': 'features'}),
    (Cast, [np.float32], {'which_sources': 'features'}) ,
    (Flatten, [], {'which_sources': 'features'}) )

cifar_train = CIFAR10(("train",), subset=slice(None,40000))
cifar_train_stream = DataStream.default_stream(
    cifar_train, iteration_scheme=ShuffledScheme(cifar_train.num_examples, 100))
                                               
cifar_validation = CIFAR10(("train",), subset=slice(40000, None))
cifar_validation_stream = DataStream.default_stream(
    cifar_validation, iteration_scheme=SequentialScheme(cifar_validation.num_examples, 250))

cifar_test = CIFAR10(("test",))
cifar_test_stream = DataStream.default_stream(
    cifar_test, iteration_scheme=SequentialScheme(cifar_test.num_examples, 250))

In [3]:
from modules.Network import Network
from modules.layers.SoftMaxLayer import SoftMaxLayer
from modules.layers.AffineLayer import AffineLayer
from modules.layers.ConvolutionalPoolLayer import ConvolutionalPoolLayer

In [4]:
net = Network([ConvolutionalPoolLayer(image_shape=(3,32,32), filter_shape=(10,3,5,5), pool_shape=(4,4)),
               AffineLayer(10*7*7, 200),
               SoftMaxLayer(200, 10)])
net.train(cifar_train_stream, cifar_validation_stream, 2, 1e-1, 0.9, 1e-3)

At batch #100, batch cost: 2.747987
At batch #200, batch cost: 2.595502
At batch #300, batch cost: 2.457380
At batch #400, batch cost: 2.446375
After epoch 1: validation error: 89.780000%
At batch #500, batch cost: 2.519971
At batch #600, batch cost: 2.399290
At batch #700, batch cost: 2.374401
At batch #800, batch cost: 2.458738
After epoch 2: validation error: 90.480000%
